# Notes

We provide this notebook for inference and visualizations. 

You can either load images from a dataloader(see Sec. 1) or from a local path(see Sec. 2).

Welcome to join [IDEA](https://idea.edu.cn/en)([中文网址](https://idea.edu.cn/))!

In [1]:
# !conda run -n dino python
# !conda activate dino

import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset




from util.visualizer import COCOVisualizer
from util import box_ops

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

/home/kartik_anand/scratch/miniconda3/envs/detr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 0. Initialize and Load Pre-trained Models

In [2]:
model_config_path = "/home/kartik_anand/scratch/kstyles/miccai/NEGRONI_CHECKPOINTS/source_only/Focalnet/sim10k/config_cfg.py" # change the path of the model config file
model_checkpoint_path = "/home/kartik_anand/scratch/kstyles/miccai/NEGRONI_CHECKPOINTS/source_only/Focalnet/sim10k/checkpoint0021.pth" # change the path of the model checkpoint
# See our Model Zoo section in README.md for more details about our pretrained models.

In [3]:
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

_IncompatibleKeys(missing_keys=['norm1.weight', 'norm1.bias', 'norm2.weight', 'norm2.bias', 'norm3.weight', 'norm3.bias'], unexpected_keys=['norm.weight', 'norm.bias'])


In [4]:
# load coco names
# with open('util/coco_id2name.json') as f:
#     id2name = json.load(f)
#     id2name = {int(k):v for k,v in id2name.items()}

# print(id2name)
id2name = {0:'car'}
print(id2name)

{0: 'car'}


# 1. Visualize images from a dataloader
## 1.1 Load Datasets

In [5]:
args.dataset_file = 'coco'

args.coco_path = "/home/kartik_anand/scratch/kstyles/miccai/NEGRONI_DATASETS/Natural/cityscapes/cityscapes_target"
args.fix_size = False


# print(args)
dataset_test = build_dataset(image_set='val', args=args)   
# dataset_train = build_dataset(image_set='train', args=args)   


data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [6]:
len(dataset_test)

500

In [7]:
from util.visualizer import COCOVisualizer
vslzr = COCOVisualizer()
postprocessors['bbox'].nms_iou_threshold = 0.25
for i in range(len(dataset_test)):
    image, targets = dataset_test[i]
    output = model.cuda()(image[None].cuda())
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
    thershold =  0.015 # set a thershold
    scores = output['scores']
    labels = output['labels']
    boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
    
    select_mask = scores > thershold
    # import ipdb; ipdb.set_trace()
    # print(boxes[select_mask].shape, dataset_test.coco.imgs[image_id]['file_name'])
#     import pdb
#     pdb.set_trace()
    # box_labels = ["pred" for item in labels[select_mask]] + ["gt" for item in targets['labels']]
    # import ipdb; ipdb.set_trace()
    # box_labels = ["pred" for item in labels[select_mask]]
    image_id = int(targets["image_id"])  # Assuming you need this as an integer for other purposes
    image_filename = (dataset_test.coco.imgs[image_id]['file_name'])  # Get the image filename as a string
    
    car_boxes = boxes[select_mask][labels[select_mask]==0]
    pred_dict = {
        # 'boxes': torch.cat((boxes[select_mask].cpu(),targets['boxes'])),
        # 'boxes': boxes[select_mask].cpu(),
        'boxes': car_boxes.cpu(),
        'size': targets['size'],
        # 'box_label': box_labels,
        'image_id': image_id,
        'filename': dataset_test.coco.imgs[image_id]['file_name']
    }
    #  Get the original image filename from the test dataset
    image_id = int(targets["image_id"])
    if image_id not in dataset_test.coco.imgs:
        print("Invalid image_id:", image_id)
        continue
    image_filename = dataset_test.coco.imgs[image_id]['file_name']

    # Print the original image filename
    # print("Original Image Filename:", image_filename)
    vslzr.visualize(image, pred_dict, savedir="/home/kartik_anand/scratch/kstyles/miccai/NEGRONI_CHECKPOINTS/visualizations/sim2city/source_only", show_in_console=False)

/DATA/scratch/kartik_anand/kstyles/miccai/SFDA/Proposed/FocalNet_DINO/models/dino/position_encoding.py:95: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_tx = self.temperatureW ** (2 * (dim_tx // 2) / self.num_pos_feats)
/DATA/scratch/kartik_anand/kstyles/miccai/SFDA/Proposed/FocalNet_DINO/models/dino/position_encoding.py:99: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use t

n
n
nn
n## 1.2 Get an Example and Visualize it

In [8]:
from util.visualizer import COCOVisualizer
from PIL import Image

vslzr = COCOVisualizer()

savedir = "/home/tajamul/scratch/FocalNet/Focalnet_expts/RSNA_New/k_fold_0/visualization_16"

for i in range(len(dataset_test)):
    image, targets = dataset_test[i]
    
    output = model.cuda()(image[None].cuda())
    output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]
    threshold = 0.1  # set a threshold
    scores = output['scores']
    labels = output['labels']
    boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
    select_mask = scores > threshold

    # Check if there are any predictions for this image
    # if not select_mask.any():
    #     print("No prediction made for Image", i)
    #     continue

    box_labels = ["pred" for item in labels[select_mask]] + ["gt" for item in targets['labels']]
    pred_dict = {
        'boxes': torch.cat((boxes[select_mask].cpu(), targets['boxes'])),
        'size': targets['size'],
        'box_label': box_labels,
        'image_id': i
    }

    # Get the original image filename from the test dataset
    image_id = int(targets["image_id"])
    if image_id not in dataset_test.coco.imgs:
        print("Invalid image_id:", image_id)
        continue
    image_filename = dataset_test.coco.imgs[image_id]['file_name']

    # Print the original image filename
    # print("Original Image Filename:", image_filename)

    # Visualize the image
    visualized_image = vslzr.visualize(image, pred_dict, show_in_console=False)

    # Check if visualization is successful
    # if visualized_image is None:
    #     print("Failed to visualize Image", i)
    #     continue

    # Save the image with the original filename
    save_path = os.path.join(savedir, image_filename)
    visualized_image.save(save_path)


AttributeError: 'NoneType' object has no attribute 'save'

In [ ]:
image, targets = dataset_val[2]
print(targets, image.shape)
# build gt_dict for vis
box_label = [id2name[int(item)] for item in targets['labels']]
gt_dict = {
    'boxes': targets['boxes'],
    'image_id': targets['image_id'],
    'size': targets['size'],
    'box_label': box_label,
}
vslzr = COCOVisualizer()
vslzr.visualize(image, gt_dict, savedir=None)

In [ ]:
image[None].cuda().shape

## 1.3 Visualize Model Predictions

In [ ]:
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
thershold = 0.3 # set a thershold
scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold
# print(scores, select_mask)
print(len(scores))

In [ ]:
box_label = ["pred" for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': targets['size'],
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None)

# 2. Visualize Custom Images

In [ ]:
from PIL import Image
import datasets.transforms as T

In [ ]:
image = Image.open("./figs/idea.jpg").convert("RGB") # load image

In [ ]:
# transform images
transform = T.Compose([
    T.RandomResize([800], max_size=1333),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
image, _ = transform(image, None)

In [ ]:
# predict images
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
# visualize outputs
thershold = 0.3 # set a thershold

vslzr = COCOVisualizer()

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': torch.Tensor([image.shape[1], image.shape[2]]),
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None, dpi=100)